In [1]:
using LinearAlgebra, Plots, Printf, Latexify, LaTeXStrings, BenchmarkTools, SparseArrays, LinearOperators, Random
using MatrixMarket: mmread
using IncompleteLU: ilu
using IterativeSolvers: gmres

In [2]:
function RR_Davidson_sym(A, 
                         q::Vector{Float64}, 
                         k::Int, 
                         tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  B = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  DA = spdiagm(diag(A, 0))
  res = 1.; rho = 1.
  while (res > tol && j < k)
    for i in 1:j
      h = Q[:, i]'t
      t .-= h .* Q[:, i]
    end
    Q[:, j+1] = t ./ norm(t) 
    j += 1
    W[:, j] = A * Q[:, j]
    B[1:j, j] = Q[:, 1:j]'W[:, j]
    B[j, 1:j-1] = Q[:, j]'W[:, 1:j-1]
    vals, vecs = eigen(B[1:j, 1:j], sortby=norm)
    rho = vals[j]
    y .= Q[:, 1:j] * vecs[:, j]
    r .= W[:, 1:j] * vecs[:, j] - rho .* y
    res = norm(r) / norm(rho)
    t .= (DA - rho * I) \ r
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [3]:
n = 400_000;
Random.seed!(1);
A = spdiagm(-3 => rand(n-3),
            -2 => rand(n-2),
            -1 => rand(n-1),
             0 => rand(n),
             1 => rand(n-1),
             2 => rand(n-2),
             3 => rand(n-3))
A = A + A';
k = 100;
tol = 1e-8;

In [4]:
res, val, y = RR_Davidson_sym(A, rand(n), k, tol);

1 0.5623342655194349 rho = 5.5025042176074805
2 0.1284415373003457 rho = 7.129215157698077
3 0.08353032086902117 rho = 7.309344587438295
4 0.08764699824055568 rho = 7.472717002187027
5 0.0776641431800263 rho = 7.612319098726835
6 0.07631265570321319 rho = 7.742804800501181
7 0.07346630946266092 rho = 7.861734448417484
8 0.07232020581705179 rho = 7.9814689105623655
9 0.06900933079867855 rho = 8.09093199631315
10 0.06538851955457156 rho = 8.194397160438847
11 0.060352732450336094 rho = 8.28353311565387
12 0.05546433321143787 rho = 8.361549823227614
13 0.05046331291828853 rho = 8.426372156629713
14 0.046222130125136686 rho = 8.481369118132104
15 0.042957645528949116 rho = 8.528137572681425
16 0.04010419317074076 rho = 8.569579597617865
17 0.03738049224492895 rho = 8.605760798157572
18 0.034010550971681965 rho = 8.637110569011542
19 0.030122445612559945 rho = 8.662455000389603
20 0.02591615457530581 rho = 8.681989223870302
21 0.021943744604167422 rho = 8.696215144695126
22 0.01842453356306

In [5]:
function RR_GD_sym(A, 
                   q::Vector{Float64}, 
                   k::Int, 
                   tol::Float64;
                   T=I)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  B = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  res = 1.; rho = 1.
  while (res > tol && j < k)
    for i in 1:j
      h = Q[:, i]'t
      t .-= h .* Q[:, i]
    end
    Q[:, j+1] = t ./ norm(t) 
    j += 1
    W[:, j] = A * Q[:, j]
    B[1:j, j] = Q[:, 1:j]'W[:, j]
    B[j, 1:j-1] = Q[:, j]'W[:, 1:j-1]
    vals, vecs = eigen(B[1:j, 1:j], sortby=norm)
    rho = vals[j]
    y .= Q[:, 1:j] * vecs[:, j]
    r .= W[:, 1:j] * vecs[:, j] - rho .* y
    res = norm(r) / norm(rho)
    t .= T \ r
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [6]:
function power_iters(A, x, it=4)
  val = 1.
  for i in 1:it
    x ./= norm(x)
    Ax = A * x
    val = x'Ax
    x .= Ax
  end
  return val
end;

Random.seed!(1);
val = power_iters(A, rand(n), 10);
P = ilu(A - val * I);

res, val, y = RR_GD_sym(A, rand(n), k, tol, T=P);

1 0.5652921354850687 rho = 5.502991844091799
2 0.0004564739385947983 rho = 7.614679111888191
3 0.0010429837013429507 rho = 7.614716169746169
4 0.0008114505698087089 rho = 7.6148277006865
5 0.0004154993029105994 rho = 7.614848274492811
6 0.0006019638798012076 rho = 7.614849160463414
7 0.0014115101560862092 rho = 7.614995047217575
8 0.0011658296432102498 rho = 7.6149998574321724
9 0.0013103667788831959 rho = 7.615076169844155
10 0.0012103880817764133 rho = 7.615077148245075
11 0.00237746016289937 rho = 7.615182809981137
12 0.0017570752449538188 rho = 7.615332087059788
13 0.0027012269515586146 rho = 7.615456597489421
14 0.0023159834847760433 rho = 7.615685092232995
15 0.002771640797325327 rho = 7.6157103198756575
16 0.0024328411154661807 rho = 7.615930030415946
17 0.002704471297357363 rho = 7.615938653385452
18 0.002500340030988222 rho = 7.6161090224985175
19 0.0026073262755803082 rho = 7.616110664942767
20 0.0028698688529993784 rho = 7.6164140100091355
21 0.0037376783967447954 rho = 7.61

In [7]:
function HR_Davidson_sym(A, 
                         q::Vector{Float64}, 
                         k::Int, 
                         sigma::Float64, 
                         tol::Float64)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  H = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  DA = spdiagm(diag(A, 0))
  res = 1.; rho = 1.
  while (res > tol && j < k)
    j += 1
    w = (A - sigma * I) * t
    for i in 1:j 
      witw = W[:, i]'w
      w .-= witw * W[:, i]
      t .-= witw * Q[:, i]
    end
    w_norm = norm(w)
    W[:, j] = w / w_norm
    Q[:, j] = t / w_norm      
    H[j, 1:j] = W[:, j]'Q[:, 1:j]
    H[1:j, j] = W[:, 1:j]'Q[:, j]
    vals, vecs = eigen(H[1:j, 1:j])
    vals = 1. ./ vals
    ind = argmin(norm.(vals))
    y .= Q[:, 1:j] * vecs[:, ind]
    yty = y'y
    y ./= sqrt(yty)
    drho = 1. / (vals[ind] * yty)
    rho = sigma + drho
    r .= W[:, 1:j] * vecs[:, ind] / sqrt(yty) - drho * y
    res = norm(r) / norm(rho)
    t .= (DA - rho * I) \ r
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [8]:
sigma = 2.;
Random.seed!(1);
res, val, y = HR_Davidson_sym(A, rand(n), k, sigma, tol);

1 0.5216274696252504 rho = 5.754506744294758
2 5.207422833772338 rho = -0.35041663590349614
3 1.751702319956281 rho = -0.8447805228684984
4 73.91980975631617 rho = 0.022978058389639067
5 73.9217541048969 rho = 0.022977407121740345
6 73.92083350112044 rho = 0.02297769291496543
7 73.86482751744754 rho = 0.02299506277166885
8 73.92716095700938 rho = 0.02297553378178896
9 73.95467337606122 rho = 0.022966955001164857
10 73.94741909872266 rho = 0.022969122076255788
11 73.93499833704101 rho = 0.022972962126049268
12 73.93607207408535 rho = 0.022972627675061696
13 73.75223017938532 rho = 0.023029714849970073
14 73.74826709040514 rho = 0.023030948055970724
15 73.6925329386231 rho = 0.02304834511551812
16 73.69649917559141 rho = 0.023047091774109907
17 73.66817696560311 rho = 0.023055947644672337
18 73.67280344337838 rho = 0.023054480394906207
19 73.59699665841377 rho = 0.023078229951535967
20 73.60943012864344 rho = 0.0230742708785463
21 73.613852940564 rho = 0.02307283342356281
22 73.531162382

In [9]:
function HR_GD_sym(A, 
                   q::Vector{Float64}, 
                   k::Int, 
                   sigma::Float64, 
                   tol::Float64;
                   T=I)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  H = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  res = 1.; rho = 1.
  while (res > tol && j < k)
    j += 1
    w = (A - sigma * I) * t
    for i in 1:j 
      witw = W[:, i]'w
      w .-= witw * W[:, i]
      t .-= witw * Q[:, i]
    end
    w_norm = norm(w)
    W[:, j] = w / w_norm
    Q[:, j] = t / w_norm      
    H[j, 1:j] = W[:, j]'Q[:, 1:j]
    H[1:j, j] = W[:, 1:j]'Q[:, j]
    vals, vecs = eigen(H[1:j, 1:j])
    vals = 1. ./ vals
    ind = argmin(norm.(vals))
    y .= Q[:, 1:j] * vecs[:, ind]
    yty = y'y
    y ./= sqrt(yty)
    drho = 1. / (vals[ind] * yty)
    rho = sigma + drho
    r .= W[:, 1:j] * vecs[:, ind] / sqrt(yty) - drho * y
    res = norm(r) / norm(rho)
    t .= T \ r
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [10]:
sigma = 2.;
P = ilu(A - sigma * I);

Random.seed!(1);
res, val, y = HR_GD_sym(A, rand(n), k, sigma, tol, T=P);

1 0.5216274696252504 rho = 5.754506744294758
2 0.0038650683397191033 rho = 1.9999508688776502
3 5.3244384752941755e-6 rho = 1.9999811874462514
4 2.3799647481973046e-6 rho = 1.9999820396357997
5 2.004744252775856e-6 rho = 1.9999839750160375
6 7.189217607108482e-7 rho = 1.9999847004451703
7 1.2172513393817337e-7 rho = 1.999984712315548
8 7.090976830598623e-8 rho = 1.9999847153196997
9 2.8119364224033014e-8 rho = 1.9999847167865226
10 3.83615710946413e-9 rho = 1.999984716850456


In [11]:
function mul_tA(A, lbda, y, x, tAx)
  c = y'x
  tAx[:] = x - c * y
  tAx[:] = (A - lbda * I) * tAx
  c = y'tAx
  tAx[:] -= c * y
end;

mutable struct OrthoPrecond
  T
  y::Vector{Float64}
end;

function LinearAlgebra.ldiv!(M::OrthoPrecond, R)
  u = M.T \ M.y
  V = M.T \ R
  alpha = - M.y'V / M.y'u
  R[:] = u * alpha + V
end;

function LinearAlgebra.ldiv!(Z, M::OrthoPrecond, R)
  ldiv!(M, R)
  Z .= R
end;

In [12]:
function RR_JD_sym(A, 
                   q::Vector{Float64}, 
                   k::Int, 
                   tol::Float64;
                   T=I,
                   gmres_iters=10)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  B = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  if T != I
    M = OrthoPrecond(T, y)
  end
  res = 1.; rho = 1.
  while (res > tol && j < k)
    for i in 1:j
      h = Q[:, i]'t
      t .-= h .* Q[:, i]
    end
    Q[:, j+1] = t ./ norm(t)
    j += 1
    W[:, j] = A * Q[:, j]
    B[1:j, j] = Q[:, 1:j]'W[:, j]
    B[j, 1:j-1] = Q[:, j]'W[:, 1:j-1]
    vals, vecs = eigen(B[1:j, 1:j], sortby=norm)
    rho = vals[j]
    y .= Q[:, 1:j] * vecs[:, j]
    r .= W[:, 1:j] * vecs[:, j] - rho .* y
    res = norm(r) / norm(rho)    
    tA = LinearOperator(Float64, n, n, false, false,
                        (tAx, x) -> mul_tA(A, rho, y, x, tAx),
                        nothing, nothing)
    if T == I
      t = gmres(tA, -r, maxiter=gmres_iters)
    else
      M.y =  y
      t = gmres(tA, -r, Pl=M, maxiter=gmres_iters)
    end
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [13]:
Random.seed!(1);
val = power_iters(A, rand(n));
P = ilu(A - val * I);

res, val, y = RR_JD_sym(A, rand(n), k, tol, T=P);

1 0.5652921354850687 rho = 5.502991844091799
2 0.0033175957149939113 rho = 7.391537105460751
3 0.002057798555631144 rho = 7.391706673521782
4 0.0007844649546339336 rho = 7.39173814131558
5 0.0005358775162960447 rho = 7.391762723064658
6 0.001186132296158454 rho = 7.391903942187004
7 0.0010817779356960312 rho = 7.391919900326394
8 0.0019331629957956655 rho = 7.391956871252985
9 0.0019027150798482722 rho = 7.392128111548407
10 0.0015949167705126104 rho = 7.392279931046083
11 0.0017076087543885549 rho = 7.392544616475645
12 0.001564985361676476 rho = 7.392548751479266
13 0.004832821701236848 rho = 7.393159777786551
14 0.004171303496598973 rho = 7.393409455410072
15 0.0041367498428043345 rho = 7.393409534999574
16 0.005980062412619207 rho = 7.394244101027532
17 0.005749784801209879 rho = 7.394415111371908
18 0.006058498665077307 rho = 7.395017806928446
19 0.005726430826767345 rho = 7.395141061017928
20 0.005424564989312227 rho = 7.39561751255829
21 0.005642360713682617 rho = 7.395634747201

In [14]:
function HR_JD_sym(A, 
                   q::Vector{Float64}, 
                   k::Int, 
                   sigma::Float64,
                   tol::Float64;
                   T=I,
                   gmres_iters=10)
  n, _ = size(A)
  Q = zeros(Float64, (n, k))
  W = zeros(Float64, (n, k))
  H = zeros(Float64, (k, k))
  y = zeros(Float64, n)
  r = zeros(Float64, n)
  t = zeros(Float64, n)
  t .= q
  j = 0
  if T != 0
    M = OrthoPrecond(T, y)
  end
  res = 1.; rho = 1.
  while (res > tol && j < k)
    j += 1
    w = (A - sigma * I) * t
    for i in 1:j 
      witw = W[:, i]'w
      w .-= witw * W[:, i]
      t .-= witw * Q[:, i]
    end
    w_norm = norm(w)
    W[:, j] = w / w_norm
    Q[:, j] = t / w_norm      
    H[j, 1:j] = W[:, j]'Q[:, 1:j]
    H[1:j, j] = W[:, 1:j]'Q[:, j]
    vals, vecs = eigen(H[1:j, 1:j])
    vals = 1. ./ vals
    ind = argmin(norm.(vals))
    y .= Q[:, 1:j] * vecs[:, ind]
    yty = y'y
    y ./= sqrt(yty)
    drho = 1. / (vals[ind] * yty)
    rho = sigma + drho
    r .= W[:, 1:j] * vecs[:, ind] / sqrt(yty) - drho * y
    res = norm(r) / norm(rho)
    tA = LinearOperator(Float64, n, n, false, false,
                        (tAx, x) -> mul_tA(A, rho, y, x, tAx),
                        nothing, nothing)
    if T == I
      t = gmres(tA, -r, maxiter=gmres_iters)
    else
      M.y =  y
      t = gmres(tA, -r, Pl=M, maxiter=gmres_iters)
    end
    println(j, ' ', res, " rho = ", rho)
  end
  return res, rho, y
end;

In [15]:
sigma = 2.;
P = ilu(A - sigma * I);

Random.seed!(1);
res, val, y = HR_JD_sym(A, rand(n), k, sigma, tol, T=P);

1 0.5216274696252504 rho = 5.754506744294758
2 0.014267120749643437 rho = 1.9995251745301632
3 7.697774449473055e-5 rho = 1.9998485936854165
4 1.911987835945399e-5 rho = 1.999851026701978
5 1.8032694842696706e-5 rho = 1.9998499907841842
6 1.8221443245487095e-5 rho = 1.9998859692691207
7 3.5233097961482445e-6 rho = 1.9998854180449723
8 8.253545132492748e-6 rho = 1.9998883135381198
9 3.1838760801042937e-6 rho = 1.999893420069769
10 3.126585964313475e-6 rho = 1.999894332670403
11 3.857042083184614e-6 rho = 1.9998971693108225
12 2.1115587974583106e-6 rho = 1.9998981296555
13 5.466784044694908e-7 rho = 1.9998985961257225
14 1.1180512548340814e-7 rho = 1.9998985949638861
15 2.1907769592383254e-8 rho = 1.999898595402957
16 4.384128117250179e-9 rho = 1.9998985953969408
